### Setup Environment

In [1]:
import os
os.environ["S3_ENDPOINT_URL"]="minio:9000"
os.environ["AWS_ACCESS_KEY_ID"]="lwycuW6S5f7yJZt65tRK"
os.environ["AWS_SECRET_ACCESS_KEY"]="d6hXquiXGpbmfR8OdX7Byd716hmhN87xTyCX8S0K"
os.environ["MLRUN_DBPATH"]="http://mlrun-api:8080"
import mlrun

In [2]:
# Set the environment:
project_name='mnist-training'
mlrun.set_environment(env_file='mlrun.env', artifact_path=f's3://mlrun/projects/{project_name}/artifacts')
#mlrun.set_environment('http://mlrun-api:8080', env_file='mlrun.env', artifact_path='s3://mlrun')

('default', 's3://mlrun/projects/mnist-training/artifacts')

### Create Project

In [3]:
# Create the project:
project_dir = os.path.abspath('./')
project = mlrun.get_or_create_project(project_name, project_dir, user_project=False)
print(project_dir)

> 2024-05-13 17:01:06,746 [info] Project loaded successfully: {'project_name': 'mnist-training'}
/home/jovyan/mnist_demo


### Create Training Function

In [4]:
# Create the training function.
trainer = project.set_function(
    "mnist_training_with_mlrun.py", name="trainer", kind="job", 
    image="mlrun/mlrun", 
    requirements=["minio", "torch", "torchvision"],
    handler="train_model"
)
print(type(trainer))

<class 'mlrun.runtimes.kubejob.KubejobRuntime'>


### Run the function

In [5]:
# Sample hyperparameters
training_parameters = {
    'batch_size': 32,
    'device': 'cpu',
    'dropout_input': 0.2,
    'dropout_hidden': 0.5,
    'epochs': 2,
    'input_size': 784,
    'hidden_sizes': [1024, 1024, 1024, 1024],
    'lr': 0.025,
    'momentum': 0.5,
    'output_size': 10,
    'smoke_test_size': -1
    }

In [6]:
# Run the function.
trainer_run = project.run_function(
    "trainer",
    inputs={"bucket_name": "mnist", "loader_type": "memory"},
    params={"training_parameters": training_parameters},
    local=True
)

> 2024-05-13 17:01:06,780 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-13 17:01:06,781 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-13 17:01:06,791 [info] Storing function: {'name': 'trainer-train-model', 'uid': 'b7a98acb8e7a4fc7b785272af533172d', 'db': 'http://mlrun-api:8080'}
> 2024-05-13 17:01:06,829 [debug] Starting local run: /home/jovyan/mnist_demo/mnist_training_with_mlrun.py # train_model
> 2024-05-13 17:01:09,514 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-05-13 17:01:09,516 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-13 17:01:09,516 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-

An error occurred (InvalidAccessKeyId) when calling the ListObjects operation: The AWS Access Key Id you provided does not exist in our records.


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mnist-training,...f533172d,0,May 13 17:01:06,error,trainer-train-model,kind=localowner=jovyanhost=e9436980ce9c,bucket_nameloader_type,"training_parameters={'batch_size': 32, 'device': 'cpu', 'dropout_input': 0.2, 'dropout_hidden': 0.5, 'epochs': 2, 'input_size': 784, 'hidden_sizes': [1024, 1024, 1024, 1024], 'lr': 0.025, 'momentum': 0.5, 'output_size': 10, 'smoke_test_size': -1}",,


> 2024-05-13 17:01:10,003 [info] Run execution finished: {'status': 'error', 'name': 'trainer-train-model'}


RunError: An error occurred (InvalidAccessKeyId) when calling the ListObjects operation: The AWS Access Key Id you provided does not exist in our records.